## Compare likelihoods for theories with massive neutrinos

We redo the notebook full_likelihood_example, but now varying also mnu. We will play with the different models in the paper on neutrinos: https://arxiv.org/abs/1911.09596

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 140
mpl.rcParams['figure.dpi'] = 140
# our modules
import data_MPGADGET
import p1d_arxiv
import gp_emulator
import likelihood
import likelihood_parameter
import lya_theory
import full_theory
import camb_cosmo
import CAMB_model
import fit_linP

### Setup mock data, from one of simulations in the suite

In [ ]:
# specify simulation suite and P1D mesurements
basedir="/p1d_emulator/sim_suites/Australia20/"
skewers_label='Ns500_wM0.05'
p1d_label=None
# specify simulation to use to generate synthetic data
test_sim_num=15
data=data_MPGADGET.P1D_MPGADGET(sim_number=test_sim_num,basedir=basedir,skewers_label=skewers_label,data_cov_factor=1)

### Setup archive of P1D measurements from simulations and train the emulator

In [ ]:
# specify redshift range
z_max=4
# do not use test_sim_num that was used in generating mock data
arxiv=p1d_arxiv.ArxivP1D(basedir=basedir,drop_sim_number=test_sim_num,
                            drop_tau_rescalings=True,z_max=z_max,
                            drop_temp_rescalings=True,skewers_label=skewers_label)

In [ ]:
# specify parameters to be used in emulator
paramList=['mF', 'sigT_Mpc', 'gamma', 'kF_Mpc', 'Delta2_p', 'n_p']
# specify k range
kmax_Mpc=8
# setup GP emulator
emu=gp_emulator.GPEmulator(basedir,p1d_label,skewers_label,z_max=z_max,
                                verbose=False,paramList=paramList,train=True,
                                emu_type="k_bin",passArxiv=arxiv,kmax_Mpc=kmax_Mpc,
                                drop_tau_rescalings=True,
                                drop_temp_rescalings=True)

### Setup relevant cosmology objects

In [5]:
# massless cosmology (to be used as fiducial)
cosmo_mnu0=camb_cosmo.get_cosmology(As=1.952e-9,mnu=0.0)
camb_cosmo.print_info(cosmo_mnu0)
# compute linP parameters
linP_mnu0=fit_linP.LinearPowerModel(cosmo=cosmo_mnu0)
old_like_params_mnu0=linP_mnu0.get_likelihood_parameters()
print('old likelihood parameters')
for par in old_like_params_mnu0:
    print(par.info_str())
# setup camb model (needed in full_theory)
camb_model_mnu0=CAMB_model.CAMBModel(zs=data.z,cosmo=cosmo_mnu0)
new_like_params_mnu0=camb_model_mnu0.get_likelihood_parameters()
print('new likelihood parameters')
for par in new_like_params_mnu0:
    print(par.info_str())

H0 = 6.7000E+01, Omega_b h^2 = 2.2000E-02, Omega_c h^2 = 1.2000E-01, Omega_k = 0.0000E+00, Omega_nu h^2 = 0.0000E+00, A_s = 1.9520E-09, n_s = 9.6500E-01, alpha_s = 0.0000E+00
old likelihood parameters
g_star = 0.9677508579459803
f_star = 0.981369557840318
Delta2_star = 0.3347920795805195
n_star = -2.29933566725751
alpha_star = -0.21652703712064045
new likelihood parameters
ombh2 = 0.022
omch2 = 0.12
As = 1.952e-09
ns = 0.965
H0 = 67.0
mnu = 0.0


In [6]:
# massive cosmology (to be used as test)
cosmo_mnu03=camb_cosmo.get_cosmology(As=2.142e-9,mnu=0.3)
camb_cosmo.print_info(cosmo_mnu03)
# compute linP parameters
linP_mnu03=fit_linP.LinearPowerModel(cosmo=cosmo_mnu03)
old_like_params_mnu03=linP_mnu03.get_likelihood_parameters()
print('old likelihood parameters')
for par in old_like_params_mnu03:
    print(par.info_str())
# setup camb model (needed in full_theory)
camb_model_mnu03=CAMB_model.CAMBModel(zs=data.z,cosmo=cosmo_mnu03)
new_like_params_mnu03=camb_model_mnu03.get_likelihood_parameters()
print('new likelihood parameters')
for par in new_like_params_mnu03:
    print(par.info_str())

H0 = 6.7000E+01, Omega_b h^2 = 2.2000E-02, Omega_c h^2 = 1.2000E-01, Omega_k = 0.0000E+00, Omega_nu h^2 = 3.2257E-03, A_s = 2.1420E-09, n_s = 9.6500E-01, alpha_s = 0.0000E+00
old likelihood parameters
g_star = 0.9687130850033866
f_star = 0.9690024397615453
Delta2_star = 0.3299425914299472
n_star = -2.30212409098578
alpha_star = -0.21455144240679364
new likelihood parameters
ombh2 = 0.022
omch2 = 0.12
As = 2.142e-09
ns = 0.965
H0 = 67.0
mnu = 0.3004409523855098


### Setup and compare theory objects for massless cosmology

Start by setting up an old theory object, that uses only compressed linP parameters

In [10]:
old_theory=lya_theory.LyaTheory(zs=data.z,emulator=emu,cosmo_fid=cosmo_mnu0,verbose=True)
old_emu_calls_mnu0=old_theory.get_emulator_calls()

0 2.0 model {'Delta2_p': 0.5855050525850493, 'n_p': -2.2993906458284763, 'alpha_p': -0.2164197078001484, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
1 2.25 model {'Delta2_p': 0.5018730987522522, 'n_p': -2.2993815689130654, 'alpha_p': -0.21642424974102917, 'f_p': 0.9666989020097663, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13051263566195997, 'kF_Mpc': 9.574544756852742}
2 2.5 model {'Delta2_p': 0.4346637488391917, 'n_p': -2.2993726176961213, 'alpha_p': -0.21642898800316582, 'f_p': 0.9729433890617771, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12986720978610813, 'kF_Mpc': 9.91377946463774}
3 2.75 model {'Delta2_p': 0.3799222893571058, 'n_p': -2.2993635887568744, 'alpha_p': -0.21643337734531473, 'f_p': 0.9776963475296397, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.1290634736772292, 'kF_Mpc': 10.258134266642541}
4 3.

Setup now a new theory object, that holds a CAMB object

In [11]:
new_theory=full_theory.FullTheory(zs=data.z,emulator=emu,camb_model_fid=camb_model_mnu0,verbose=True)
new_emu_calls_mnu0=new_theory.get_emulator_calls(return_M_of_z=False)

0 2.0 model {'Delta2_p': 0.5856948643323825, 'n_p': -2.2994471404944616, 'alpha_p': -0.2165293646872732, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
1 2.25 model {'Delta2_p': 0.5020491175936141, 'n_p': -2.299447102944892, 'alpha_p': -0.2165301510752391, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13051263566195997, 'kF_Mpc': 9.574544756852742}
2 2.5 model {'Delta2_p': 0.4348272639660884, 'n_p': -2.299447107345488, 'alpha_p': -0.21652926356416427, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12986720978610813, 'kF_Mpc': 9.91377946463774}
3 2.75 model {'Delta2_p': 0.3800747525040643, 'n_p': -2.299447113528971, 'alpha_p': -0.21653001056427074, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.1290634736772292, 'kF_Mpc': 10.258134266642541}
4 3.0 model {'Delta2_p': 0.3349345497970454, 'n_p': -2.29944714097795, 'alpha_p': -0.21652967998711606, 'mF': 0.6601

In [12]:
print("z=2 emulator calls for old theory = ", old_emu_calls_mnu0[0])
print("z=2 emulator calls for new theory = ", new_emu_calls_mnu0[0])

z=2 emulator calls for old theory =  {'Delta2_p': 0.5855050525850493, 'n_p': -2.2993906458284763, 'alpha_p': -0.2164197078001484, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
z=2 emulator calls for new theory =  {'Delta2_p': 0.5856948643323825, 'n_p': -2.2994471404944616, 'alpha_p': -0.2165293646872732, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}


Note that the numbers above are not supposed to exactly match, since kp_Mpc is slightly different in both theories

### Compare now emulator calls for the massive neutrino cosmology

In [14]:
# call the old theory, but using the freshly computed likelihood parameters from the test cosmology
old_emu_calls_mnu03=old_theory.get_emulator_calls(like_params=old_like_params_mnu03)

0 2.0 model {'Delta2_p': 0.5729326720344529, 'n_p': -2.302179069556749, 'alpha_p': -0.21444411308629752, 'f_p': 0.9459897955896628, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13084042738901472, 'kF_Mpc': 9.230808629264576}
1 2.25 model {'Delta2_p': 0.49206975658291824, 'n_p': -2.302169992641338, 'alpha_p': -0.21444865502718433, 'f_p': 0.9543317839309936, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13055175541956876, 'kF_Mpc': 9.571675750082983}
2 2.5 model {'Delta2_p': 0.42695513472108815, 'n_p': -2.302161041424392, 'alpha_p': -0.214453393289324, 'f_p': 0.9605762709830045, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12989224167162092, 'kF_Mpc': 9.91186895336036}
3 2.75 model {'Delta2_p': 0.3738218742441991, 'n_p': -2.302152012485143, 'alpha_p': -0.21445778263147938, 'f_p': 0.9653292294508671, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.12907549663747042, 'kF_Mpc': 10.257178754995095}
4 3.0 

In [16]:
# we want to study also cases where we ignore some of the parameters
cull_params=['f_star','g_star','alpha_star']
culled_like_params_mnu03 = [par for par in old_like_params_mnu03 if par.name not in cull_params]
for par in culled_like_params_mnu03:
    print(par.info_str())
# same, but using only the culled parameters
culled_emu_calls_mnu03=old_theory.get_emulator_calls(like_params=culled_like_params_mnu03)

Delta2_star = 0.3299425914299472
n_star = -2.30212409098578


In [21]:
# call the new theory, but using the freshly computed likelihood parameters for the test cosmology
new_emu_calls_mnu03=new_theory.get_emulator_calls(like_params=new_like_params_mnu03,return_M_of_z=False)

0 2.0 model {'Delta2_p': 0.5688959105648443, 'n_p': -2.300028997230093, 'alpha_p': -0.2155325928984313, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.12948756671119963, 'kF_Mpc': 9.327250305760211}
1 2.25 model {'Delta2_p': 0.48855637474755614, 'n_p': -2.2999913375238594, 'alpha_p': -0.21559281101966773, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.1291863101929846, 'kF_Mpc': 9.67284435644567}
2 2.5 model {'Delta2_p': 0.4238891540519851, 'n_p': -2.299957980048286, 'alpha_p': -0.2156459638190625, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12852472662513884, 'kF_Mpc': 10.017332161012323}
3 2.75 model {'Delta2_p': 0.3711355095664784, 'n_p': -2.2999286719560357, 'alpha_p': -0.2156930097415507, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.1277121705984294, 'kF_Mpc': 10.366674027201793}
4 3.0 model {'Delta2_p': 0.327577898880286, 'n_p': -2.299902502815766, 'alpha_p': -0.2157349776779857, 'mF': 0.6601

In [23]:
print("z = {} emulator calls from old theory = {}".format(data.z[0],old_emu_calls_mnu03[0]))
print("z = {} emulator calls from culled theory = {}".format(data.z[0],culled_emu_calls_mnu03[0]))
print("z = {} emulator calls from new theory = {}".format(data.z[0],new_emu_calls_mnu03[0]))

z = 2.0 emulator calls from old theory = {'Delta2_p': 0.5729326720344529, 'n_p': -2.302179069556749, 'alpha_p': -0.21444411308629752, 'f_p': 0.9459897955896628, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13084042738901472, 'kF_Mpc': 9.230808629264576}
z = 2.0 emulator calls from culled theory = {'Delta2_p': 0.5770239683904371, 'n_p': -2.302179069556748, 'alpha_p': -0.21641970780014372, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
z = 2.0 emulator calls from new theory = {'Delta2_p': 0.5688959105648443, 'n_p': -2.300028997230093, 'alpha_p': -0.2155325928984313, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.12948756671119963, 'kF_Mpc': 9.327250305760211}


In [22]:
print("z = {} emulator calls from old theory = {}".format(data.z[-1],old_emu_calls_mnu03[-1]))
print("z = {} emulator calls from culled theory = {}".format(data.z[-1],culled_emu_calls_mnu03[-1]))
print("z = {} emulator calls from new theory = {}".format(data.z[-1],new_emu_calls_mnu03[-1]))

z = 4.0 emulator calls from old theory = {'Delta2_p': 0.21364479473464218, 'n_p': -2.302109852904911, 'alpha_p': -0.21447894017286903, 'f_p': 0.9774561579941656, 'mF': 0.3667549106286143, 'gamma': 1.4366944565860078, 'sigT_Mpc': 0.1124506575662015, 'kF_Mpc': 12.845338449227931}
z = 4.0 emulator calls from culled theory = {'Delta2_p': 0.21246887626054778, 'n_p': -2.302109852904912, 'alpha_p': -0.2164545348867202, 'f_p': 0.9898232760729383, 'mF': 0.3667549106286143, 'gamma': 1.4366944565860078, 'sigT_Mpc': 0.1124868806234996, 'kF_Mpc': 12.841201989686333}
z = 4.0 emulator calls from new theory = {'Delta2_p': 0.21215645427288457, 'n_p': -2.29982179430806, 'alpha_p': -0.2158673732213262, 'mF': 0.3667549106286143, 'gamma': 1.4366944565860078, 'sigT_Mpc': 0.11127195468557453, 'kF_Mpc': 12.981409011442079}


Both new and old theories make the same emulator call for a theory that is not the fiducial!

## Test now the likelihood objects

### Old likelihood using all five parameters

In [30]:
## Set up a likelihood object with the old theory object
#old_free_params=['Delta2_star','n_star']
old_free_params=['Delta2_star','n_star','alpha_star','f_star','g_star']
old_like=likelihood.Likelihood(data=data,theory=old_theory,
                            free_parameters=old_free_params,
                            free_param_limits=None,
                            verbose=True,
                            prior_Gauss_rms=0.15,
                            emu_cov_factor=1)

got parameters
g_star = 0.9677508579459803
f_star = 0.981369557840318
Delta2_star = 0.3347920795805195
n_star = -2.29933566725751
alpha_star = -0.21652703712064045
ln_tau_0 = 0.0
ln_tau_1 = 0.0
ln_sigT_kms_0 = 0.0
ln_sigT_kms_1 = 0.0
ln_gamma_0 = 0.0
ln_gamma_1 = 0.0
ln_kF_0 = 0.0
ln_kF_1 = 0.0
likelihood setup with 5 free parameters
5 free parameters


In [33]:
# get parameter values in cube for test cosmology
old_values_mnu03=old_like.sampling_point_from_parameters(old_like_params_mnu03)
# check that parameters inferred from these are same than computed before
for par in old_like.parameters_from_sampling_point(values=old_values_mnu03):
    print(par.info_str(),',',par.value_in_cube())
old_like.get_log_like(values=old_values_mnu03,ignore_log_det_cov=False)

g_star = 0.9687130850033866 , 0.4678271250846657
f_star = 0.9690024397615453 , 0.47506099403863367
Delta2_star = 0.3299425914299472 , 0.5329506095329812
n_star = -2.30212409098578 , 0.47875909014220186
alpha_star = -0.21455144240679364 , 0.5040777963018761
0 2.0 model {'Delta2_p': 0.5729326720344529, 'n_p': -2.302179069556749, 'alpha_p': -0.21444411308629752, 'f_p': 0.9459897955896628, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13084042738901472, 'kF_Mpc': 9.230808629264576}
1 2.25 model {'Delta2_p': 0.49206975658291824, 'n_p': -2.302169992641338, 'alpha_p': -0.21444865502718433, 'f_p': 0.9543317839309936, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13055175541956876, 'kF_Mpc': 9.571675750082983}
2 2.5 model {'Delta2_p': 0.42695513472108815, 'n_p': -2.302161041424392, 'alpha_p': -0.214453393289324, 'f_p': 0.9605762709830045, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12989224167162092, 'kF_Mpc': 9.91186895336036}


-156.54341456470135

### Old likelihood using only two parameters

In [34]:
## Set up a likelihood object with the old theory object, but now with only 2 key parameters
culled_free_params=['Delta2_star','n_star']
culled_like=likelihood.Likelihood(data=data,theory=old_theory,
                            free_parameters=culled_free_params,
                            free_param_limits=None,
                            verbose=True,
                            prior_Gauss_rms=0.15,
                            emu_cov_factor=1)

got parameters
g_star = 0.9677508579459803
f_star = 0.981369557840318
Delta2_star = 0.3347920795805195
n_star = -2.29933566725751
alpha_star = -0.21652703712064045
ln_tau_0 = 0.0
ln_tau_1 = 0.0
ln_sigT_kms_0 = 0.0
ln_sigT_kms_1 = 0.0
ln_gamma_0 = 0.0
ln_gamma_1 = 0.0
ln_kF_0 = 0.0
ln_kF_1 = 0.0
likelihood setup with 2 free parameters
2 free parameters


In [38]:
# get parameter values in cube for test cosmology
culled_values_mnu03=culled_like.sampling_point_from_parameters(culled_like_params_mnu03)
# check that parameters inferred from these are same than computed before
for par in culled_like.parameters_from_sampling_point(values=culled_values_mnu03):
    print(par.info_str(),',',par.value_in_cube())
culled_like.get_log_like(values=culled_values_mnu03,ignore_log_det_cov=False)

Delta2_star = 0.3299425914299472 , 0.5329506095329812
n_star = -2.30212409098578 , 0.47875909014220186
0 2.0 model {'Delta2_p': 0.5770239683904371, 'n_p': -2.302179069556748, 'alpha_p': -0.21641970780014372, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
1 2.25 model {'Delta2_p': 0.4946034296234621, 'n_p': -2.302169992641336, 'alpha_p': -0.2164242497410278, 'f_p': 0.9666989020097663, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13051263566195997, 'kF_Mpc': 9.574544756852742}
2 2.5 model {'Delta2_p': 0.4283676121381084, 'n_p': -2.302161041424391, 'alpha_p': -0.21642898800316643, 'f_p': 0.9729433890617771, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12986720978610813, 'kF_Mpc': 9.91377946463774}
3 2.75 model {'Delta2_p': 0.3744190867643675, 'n_p': -2.302152012485144, 'alpha_p': -0.21643337734531684, 'f_p': 0.9776963475296397, 'mF': 0.71911843464902

-161.08971800717748

### New likelihood

In [39]:
## Set up a likelihood object with the new theory object
new_free_params=["As","ns","mnu"]
new_like=likelihood.Likelihood(data=data,theory=new_theory,
                            free_parameters=new_free_params,
                            free_param_limits=None,
                            verbose=True,
                            prior_Gauss_rms=0.15,
                            emu_cov_factor=1)

got parameters
ombh2 = 0.022
omch2 = 0.12
As = 1.952e-09
ns = 0.965
H0 = 67.0
mnu = 0.0
ln_tau_0 = 0.0
ln_tau_1 = 0.0
ln_sigT_kms_0 = 0.0
ln_sigT_kms_1 = 0.0
ln_gamma_0 = 0.0
ln_gamma_1 = 0.0
ln_kF_0 = 0.0
ln_kF_1 = 0.0
likelihood setup with 3 free parameters
3 free parameters


In [40]:
# get parameter values in cube for test cosmology in new theory
new_values_mnu03=new_like.sampling_point_from_parameters(new_like_params_mnu03)
# check that parameters inferred from these are same than computed before
for par in new_like.parameters_from_sampling_point(values=new_values_mnu03):
    print(par.info_str(),',',par.value_in_cube())

As = 2.142e-09 , 0.7099999999999995
ns = 0.965 , 0.5
mnu = 0.3004409523855098 , 0.3004409523855098


In [41]:
new_like.get_log_like(values=new_values_mnu03,ignore_log_det_cov=False)

0 2.0 model {'Delta2_p': 0.5688959105648443, 'n_p': -2.300028997230093, 'alpha_p': -0.2155325928984313, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.12948756671119963, 'kF_Mpc': 9.327250305760211}
1 2.25 model {'Delta2_p': 0.48855637474755614, 'n_p': -2.2999913375238594, 'alpha_p': -0.21559281101966773, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.1291863101929846, 'kF_Mpc': 9.67284435644567}
2 2.5 model {'Delta2_p': 0.4238891540519851, 'n_p': -2.299957980048286, 'alpha_p': -0.2156459638190625, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12852472662513884, 'kF_Mpc': 10.017332161012323}
3 2.75 model {'Delta2_p': 0.3711355095664784, 'n_p': -2.2999286719560357, 'alpha_p': -0.2156930097415507, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.1277121705984294, 'kF_Mpc': 10.366674027201793}
4 3.0 model {'Delta2_p': 0.327577898880286, 'n_p': -2.299902502815766, 'alpha_p': -0.2157349776779857, 'mF': 0.6601

-89.8521453599527